### task-5 Model Training and Tracking

In [1]:
import sys, os
import pandas as pd
sys.path.append(os.path.abspath(".."))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
df1 = pd.read_csv('../data/processed/final_with_risk.csv')
df1.head()

,CustomerId,total_value,avg_value,std_value,transaction_count,total_amount,avg_amount,std_amount,CurrencyCode_UGX,CountryCode_256,...,ChannelId_ChannelId_3,ChannelId_ChannelId_5,PricingStrategy_0,PricingStrategy_1,PricingStrategy_2,PricingStrategy_4,FraudResult_0,FraudResult_1,TransactionStartTime,is_high_risk
0,CustomerId_1,-0.089524,-0.052297,0.000000,-0.253459,-0.066891,-0.153364,0.000000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2018-11-21 16:49:14,1
1,CustomerId_10,-0.089524,-0.052297,0.000000,-0.253459,-0.066891,-0.153364,0.000000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2018-11-21 16:49:09,1
2,CustomerId_1001,-0.082011,-0.075710,-0.120344,-0.212186,-0.055849,-0.069870,-0.105976,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2018-11-16 08:20:39,1
3,CustomerId_1002,-0.091448,-0.109431,-0.157377,-0.150278,-0.061655,-0.091435,-0.168036,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2019-01-18 10:05:00,0
4,CustomerId_1003,-0.081422,-0.080169,-0.121945,-0.201868,-0.055849,-0.073846,-0.111444,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2019-02-01 15:04:51,0


In [ ]:
# run_training.py
from src.model_training import train_models

train_models("../data/processed/final_with_risk.csv")


🔍 Training LogisticRegression...


c:\Users\ssss\Desktop\## 10 Acadamy\Week5\credit-risk-model\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\ssss\Desktop\## 10 Acadamy\Week5\credit-risk-model\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as sho

              precision    recall  f1-score   support

           0       0.93      0.99      0.96       468
           1       0.98      0.87      0.92       281

    accuracy                           0.95       749
   macro avg       0.96      0.93      0.94       749
weighted avg       0.95      0.95      0.94       749


🔍 Training RandomForest...


2025/07/03 11:18:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/03 11:19:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


              precision    recall  f1-score   support

           0       0.98      1.00      0.99       468
           1       1.00      0.96      0.98       281

    accuracy                           0.98       749
   macro avg       0.99      0.98      0.98       749
weighted avg       0.98      0.98      0.98       749


✅ Best model: RandomForestClassifier(max_depth=20, random_state=42)


,n_estimators,100
,criterion,'gini'
,max_depth,20
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [1]:
import mlflow
from mlflow.tracking import MlflowClient

def register_best_model(best_model, model_name="RandomForest", run_id=None):
    client = MlflowClient()
    model_uri = f"runs:/{run_id}/{model_name}"

    # Safe creation (does not raise error if model exists)
    try:
        client.create_registered_model(model_name)
        print(f"✅ Registered model name created: {model_name}")
    except mlflow.exceptions.RestException as e:
        if "RESOURCE_ALREADY_EXISTS" in str(e):
            print(f"ℹ️ Model {model_name} already registered.")
        else:
            raise

    # Create a new version of the model
    model_version = client.create_model_version(
        name=model_name,
        source=model_uri,
        run_id=run_id
    )
    print(f"✅ Model version {model_version.version} registered under {model_name}")

In [ ]:
mlflow.sklearn.log_model(clf.best_estimator_, model_name)
register_best_model(clf.best_estimator_, model_name, run_id)